In [ ]:
!pip install pingouin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.6 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
import pingouin as pg

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
finalpass_path = '/content/drive/MyDrive/Finalpass_entries.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/ap_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_GT.csv'

# 1. Finalpass_entries.csv에서 'Category 2' 열의 파일 이름을 가져오기 (4자리 숫자로 매칭)
finalpass_df = pd.read_csv(finalpass_path)
finalpass_files = finalpass_df['Category 2'].str[:4]  # 파일 이름의 앞 4자리 추출

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 앞 4자리로 파일 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리

# Finalpass_entries.csv에 있는 파일만 필터링
ap_cobb_final_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(finalpass_files)]
results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(finalpass_files)]

# 4. NaN 값 처리: NaN 값이 있으면 이를 무시하고 계산
ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

# 각도 값 데이터
y_true_all = results_gt_filtered['all'].values  # GT 'all' 값
y_pred_all = ap_cobb_final_filtered['all'].values  # 예측 'all' 값

y_true_max = results_gt_filtered['max'].values  # GT 'max' 값
y_pred_max = ap_cobb_final_filtered['max'].values  # 예측 'max' 값

# 샘플 이름, GT 값, 예측 값 함께 저장할 리스트
sample_results = []
for i in range(len(y_true_all)):
    image_name = ap_cobb_final_filtered.iloc[i]['image_name']
    gt_all = y_true_all[i]
    pred_all = y_pred_all[i]
    gt_max = y_true_max[i]
    pred_max = y_pred_max[i]
    sample_results.append([image_name, gt_all, pred_all, gt_max, pred_max])

# DataFrame으로 변환하여 보기 쉽게
sample_results_df = pd.DataFrame(sample_results, columns=['Image Name', 'GT All', 'Pred All', 'GT Max', 'Pred Max'])

# 5. MAE 계산
mae_all = mean_absolute_error(y_true_all, y_pred_all)
mae_max = mean_absolute_error(y_true_max, y_pred_max)

# 6. MAD 계산 (Mean Absolute Deviation)
mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

# 7. Pearson 상관 계수
pearson_all, _ = pearsonr(y_true_all, y_pred_all)
pearson_max, _ = pearsonr(y_true_max, y_pred_max)

# 8. Spearman 상관 계수
spearman_all, _ = spearmanr(y_true_all, y_pred_all)
spearman_max, _ = spearmanr(y_true_max, y_pred_max)

# 9. MAPE 계산 (Mean Absolute Percentage Error)
mape_all = np.mean(np.abs((y_true_all - y_pred_all) / y_true_all)) * 100
mape_max = np.mean(np.abs((y_true_max - y_pred_max) / y_true_max)) * 100

# 10. R값 (결정 계수, R^2) 계산
r2_all = r2_score(y_true_all, y_pred_all)
r2_max = r2_score(y_true_max, y_pred_max)

# 결과 출력
print("0번-4번 사이 각도 비교:")
print(sample_results_df[['Image Name', 'GT All', 'Pred All']])
print(f"0번-4번 사이 각도에 대한 MAE: {mae_all:.4f}")
print(f"Pearson 상관 계수: {pearson_all:.4f}")
print(f"Spearman 상관 계수: {spearman_all:.4f}")
print(f"MAPE: {mape_all:.4f}%")
print(f"MAD: {mad_all:.4f}")
print(f"R^2 (결정 계수): {r2_all:.4f}")

print("\n최대 각도 비교:")
print(sample_results_df[['Image Name', 'GT Max', 'Pred Max']])
print(f"최대 각도에 대한 MAE: {mae_max:.4f}")
print(f"Pearson 상관 계수: {pearson_max:.4f}")
print(f"Spearman 상관 계수: {spearman_max:.4f}")
print(f"MAPE: {mape_max:.4f}%")
print(f"MAD: {mad_max:.4f}")
print(f"R^2 (결정 계수): {r2_max:.4f}")

import pandas as pd
import numpy as np

# ICC 계산
icc_data_all = pd.DataFrame({
    'target': np.repeat(range(len(y_true_all)), 2),
    'rater': np.tile(['GT', 'Pred'], len(y_true_all)),
    'rating': np.concatenate([y_true_all, y_pred_all])
})

icc_data_max = pd.DataFrame({
    'target': np.repeat(range(len(y_true_max)), 2),
    'rater': np.tile(['GT', 'Pred'], len(y_true_max)),
    'rating': np.concatenate([y_true_max, y_pred_max])
})

icc_all = pg.intraclass_corr(data=icc_data_all, targets='target', raters='rater', ratings='rating')
icc_max = pg.intraclass_corr(data=icc_data_max, targets='target', raters='rater', ratings='rating')

# ICC 결과 출력
print("\nICC for 0번-4번 사이 각도:")
print(icc_all)

print("\nICC for 최대 각도:")
print(icc_max)

# ICC 결과 출력
print("\nICC for 0번-4번 사이 각도:")
print(icc_all)

print("\nICC for 최대 각도:")
print(icc_max)

# ICC 인덱스 확인
print("\nICC 인덱스:")
print(icc_all.index)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-14-b763f3afd98b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-14-b763f3afd98b>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)


0번-4번 사이 각도 비교:
          Image Name  GT All  Pred All
0   0001-F-037Y0.jpg    1.62      1.42
1   0008-M-022Y0.jpg    1.85      0.22
2   0009-F-038Y0.jpg    7.00      9.69
3   0012-F-013Y0.jpg    6.84      7.44
4   0016-F-023Y0.jpg   16.81     17.18
5   0020-M-030Y0.jpg    8.34      6.86
6   0022-M-006Y0.jpg    3.55      0.92
7   0023-F-014Y0.jpg    2.19      6.47
8   0026-F-029Y0.jpg   16.27     17.59
9   0038-F-020Y0.jpg    7.38      8.73
10  0040-M-023Y0.jpg    2.24      1.61
11  0043-F-039Y0.jpg    6.36      9.06
12  0046-F-021Y0.jpg   22.24     18.99
13  0057-F-030Y0.jpg    0.98      1.45
14  0059-F-028Y0.jpg    1.02      1.25
15  0062-F-041Y0.jpg   11.34      6.13
16  0065-F-032Y0.jpg    0.71      0.78
17  0067-F-028Y0.jpg    1.50      2.11
18  0073-F-036Y0.jpg    1.45      0.93
19  0109-F-042Y0.jpg   10.50     13.57
20  0112-F-064Y0.jpg    5.31      4.52
21  0138-M-064Y0.jpg    2.51      6.12
22  0139-F-077Y0.jpg   33.13     33.16
23  0140-M-024Y0.jpg    1.54      2.05
24  0141-

TypeError: argument of type 'int' is not iterable

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
finalpass_path = '/content/drive/MyDrive/Finalpass_entries.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/la_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_LA_GT.csv'

# 1. Finalpass_entries.csv에서 'Category 2' 열의 파일 이름을 가져오기 (4자리 숫자로 매칭)
finalpass_df = pd.read_csv(finalpass_path)
finalpass_files = finalpass_df['Category 2'].str[:4]  # 파일 이름의 앞 4자리 추출

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 앞 4자리로 파일 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리

# Finalpass_entries.csv에 있는 파일만 필터링
ap_cobb_final_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(finalpass_files)]
results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(finalpass_files)]

# 4. NaN 값 처리: NaN 값이 있으면 이를 무시하고 계산
ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

# 각도 값 데이터
y_true_all = results_gt_filtered['all'].values  # GT 'all' 값
y_pred_all = ap_cobb_final_filtered['all'].values  # 예측 'all' 값

y_true_max = results_gt_filtered['max'].values  # GT 'max' 값
y_pred_max = ap_cobb_final_filtered['max'].values  # 예측 'max' 값

# 샘플 이름, GT 값, 예측 값 함께 저장할 리스트
sample_results = []
for i in range(len(y_true_all)):
    image_name = ap_cobb_final_filtered.iloc[i]['image_name']
    gt_all = y_true_all[i]
    pred_all = y_pred_all[i]
    gt_max = y_true_max[i]
    pred_max = y_pred_max[i]
    sample_results.append([image_name, gt_all, pred_all, gt_max, pred_max])

# DataFrame으로 변환하여 보기 쉽게
sample_results_df = pd.DataFrame(sample_results, columns=['Image Name', 'GT All', 'Pred All', 'GT Max', 'Pred Max'])

# 5. MAE 계산
mae_all = mean_absolute_error(y_true_all, y_pred_all)
mae_max = mean_absolute_error(y_true_max, y_pred_max)

# 6. MAD 계산 (Mean Absolute Deviation)
mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

# 7. Pearson 상관 계수
pearson_all, _ = pearsonr(y_true_all, y_pred_all)
pearson_max, _ = pearsonr(y_true_max, y_pred_max)

# 8. Spearman 상관 계수
spearman_all, _ = spearmanr(y_true_all, y_pred_all)
spearman_max, _ = spearmanr(y_true_max, y_pred_max)

# 9. MAPE 계산 (Mean Absolute Percentage Error)
mape_all = np.mean(np.abs((y_true_all - y_pred_all) / y_true_all)) * 100
mape_max = np.mean(np.abs((y_true_max - y_pred_max) / y_true_max)) * 100

# 10. R값 (결정 계수, R^2) 계산
r2_all = r2_score(y_true_all, y_pred_all)
r2_max = r2_score(y_true_max, y_pred_max)

# 결과 출력
print("0번-4번 사이 각도 비교:")
print(sample_results_df[['Image Name', 'GT All', 'Pred All']])
print(f"0번-4번 사이 각도에 대한 MAE: {mae_all:.4f}")
print(f"Pearson 상관 계수: {pearson_all:.4f}")
print(f"Spearman 상관 계수: {spearman_all:.4f}")
print(f"MAPE: {mape_all:.4f}%")
print(f"MAD: {mad_all:.4f}")
print(f"R^2 (결정 계수): {r2_all:.4f}")

print("\n최대 각도 비교:")
print(sample_results_df[['Image Name', 'GT Max', 'Pred Max']])
print(f"최대 각도에 대한 MAE: {mae_max:.4f}")
print(f"Pearson 상관 계수: {pearson_max:.4f}")
print(f"Spearman 상관 계수: {spearman_max:.4f}")
print(f"MAPE: {mape_max:.4f}%")
print(f"MAD: {mad_max:.4f}")
print(f"R^2 (결정 계수): {r2_max:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0번-4번 사이 각도 비교:
          Image Name  GT All  Pred All
0   0001-F-037Y1.jpg   10.10      8.77
1   0008-M-022Y1.jpg   22.34     21.72
2   0009-F-038Y1.jpg   29.46     29.99
3   0012-F-013Y1.jpg   27.55     25.87
4   0016-F-023Y1.jpg   20.50     21.12
5   0020-M-030Y1.jpg   35.34     35.78
6   0022-M-006Y1.jpg   18.09     24.26
7   0023-F-014Y1.jpg   30.46     31.40
8   0026-F-029Y1.jpg   35.46     34.71
9   0038-F-020Y1.jpg   39.85     40.76
10  0040-M-023Y1.jpg   32.80     33.45
11  0043-F-039Y1.jpg   25.97     28.92
12  0046-F-021Y1.jpg   31.76     32.43
13  0057-F-030Y1.jpg   28.32     25.74
14  0059-F-028Y1.jpg   22.41     22.34
15  0062-F-041Y1.jpg   15.80     14.06
16  0065-F-032Y1.jpg   31.11     32.27
17  0067-F-028Y1.jpg   28.87     29.38
18  0073-F-036Y1.jpg   35.65     32.88
19  0109-F-042Y1.jpg   21.15     22.85
20  0112-F-064Y1.jpg   25.74     26.

<ipython-input-4-29f7026e8f0c>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-4-29f7026e8f0c>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
categories_path = '/content/drive/MyDrive/categories.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/la_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_LA_GT.csv'

# 1. categories.csv에서 각 카테고리별로 파일 이름을 가져오기
categories_df = pd.read_csv(categories_path)
category_columns = ['Category 1', 'Category 3', 'Category 4', 'Category 5', 'Category 6', 'Category 7']

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 파일 이름의 앞 4자리로 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]

# 카테고리별 MAE, Pearson, R2, MAD 계산 결과 저장할 딕셔너리
category_metrics = {}

# 4. 각 카테고리별로 값 계산
for category in category_columns:
    # 각 카테고리의 파일 이름 추출
    category_files = categories_df[category].dropna().str[:4]  # NaN 제거 후 파일 이름 앞 4자리 추출

    # 해당 카테고리의 파일에 해당하는 데이터 필터링
    ap_cobb_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(category_files)]
    results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(category_files)]

    # NaN 값 처리
    ap_cobb_filtered.dropna(subset=['all', 'max'], inplace=True)
    results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

    # GT 및 예측 값 데이터 추출
    y_true_all = results_gt_filtered['all'].values
    y_pred_all = ap_cobb_filtered['all'].values

    y_true_max = results_gt_filtered['max'].values
    y_pred_max = ap_cobb_filtered['max'].values

    # MAE 계산
    mae_all = mean_absolute_error(y_true_all, y_pred_all)
    mae_max = mean_absolute_error(y_true_max, y_pred_max)

    # MAD 계산
    mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
    mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

    # Pearson 상관 계수 계산
    pearson_all, _ = pearsonr(y_true_all, y_pred_all)
    pearson_max, _ = pearsonr(y_true_max, y_pred_max)

    # R^2 계산
    r2_all = r2_score(y_true_all, y_pred_all)
    r2_max = r2_score(y_true_max, y_pred_max)

    # 결과를 딕셔너리에 저장
    category_metrics[category] = {
        'MAE (All)': mae_all, 'MAE (Max)': mae_max,
        'MAD (All)': mad_all, 'MAD (Max)': mad_max,
        'Pearson (All)': pearson_all, 'Pearson (Max)': pearson_max,
        'R^2 (All)': r2_all, 'R^2 (Max)': r2_max
    }

# 카테고리별로 결과 출력
for category, metrics in category_metrics.items():
    print(f"\n카테고리: {category}")
    print(f"MAE (All): {metrics['MAE (All)']:.4f}, MAE (Max): {metrics['MAE (Max)']:.4f}")
    print(f"MAD (All): {metrics['MAD (All)']:.4f}, MAD (Max): {metrics['MAD (Max)']:.4f}")
    print(f"Pearson (All): {metrics['Pearson (All)']:.4f}, Pearson (Max): {metrics['Pearson (Max)']:.4f}")
    print(f"R^2 (All): {metrics['R^2 (All)']:.4f}, R^2 (Max): {metrics['R^2 (Max)']:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-8-688ae97545be>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-8-688ae97545be>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-8-688ae97545be>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-i

ValueError: x and y must have length at least 2.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import pearsonr

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
categories_path = '/content/drive/MyDrive/categories.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/la_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_LA_GT.csv'

# 1. categories.csv에서 각 카테고리별로 파일 이름을 가져오기
categories_df = pd.read_csv(categories_path)
category_columns = ['Category 1', 'Category 3', 'Category 4', 'Category 5', 'Category 6', 'Category 7']

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 파일 이름의 앞 4자리로 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]

# 카테고리별 MAE, Pearson, R2, MAD 계산 결과 저장할 딕셔너리
category_metrics = {}

# 4. 각 카테고리별로 값 계산
for category in category_columns:
    # 각 카테고리의 파일 이름 추출
    category_files = categories_df[category].dropna().str[:4]  # NaN 제거 후 파일 이름 앞 4자리 추출

    # 해당 카테고리의 파일에 해당하는 데이터 필터링
    ap_cobb_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(category_files)].copy()
    results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(category_files)].copy()

    # NaN 값 처리
    ap_cobb_filtered.dropna(subset=['all', 'max'], inplace=True)
    results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

    # 데이터가 2개 이상인지 확인 후 계산
    if len(results_gt_filtered) >= 2 and len(ap_cobb_filtered) >= 2:
        # GT 및 예측 값 데이터 추출
        y_true_all = results_gt_filtered['all'].values
        y_pred_all = ap_cobb_filtered['all'].values

        y_true_max = results_gt_filtered['max'].values
        y_pred_max = ap_cobb_filtered['max'].values

        # MAE 계산
        mae_all = mean_absolute_error(y_true_all, y_pred_all)
        mae_max = mean_absolute_error(y_true_max, y_pred_max)

        # MAD 계산
        mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
        mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

        # Pearson 상관 계수 계산
        pearson_all, _ = pearsonr(y_true_all, y_pred_all)
        pearson_max, _ = pearsonr(y_true_max, y_pred_max)

        # R^2 계산
        r2_all = r2_score(y_true_all, y_pred_all)
        r2_max = r2_score(y_true_max, y_pred_max)

        # 결과를 딕셔너리에 저장
        category_metrics[category] = {
            'MAE (All)': mae_all, 'MAE (Max)': mae_max,
            'MAD (All)': mad_all, 'MAD (Max)': mad_max,
            'Pearson (All)': pearson_all, 'Pearson (Max)': pearson_max,
            'R^2 (All)': r2_all, 'R^2 (Max)': r2_max
        }
    else:
        # 데이터가 2개 미만인 경우 경고 메시지 출력
        print(f"카테고리 {category}의 데이터가 충분하지 않음")

# 카테고리별로 결과 출력
for category, metrics in category_metrics.items():
    print(f"\n카테고리: {category}")
    print(f"MAE (All): {metrics['MAE (All)']:.4f}, MAE (Max): {metrics['MAE (Max)']:.4f}")
    print(f"MAD (All): {metrics['MAD (All)']:.4f}, MAD (Max): {metrics['MAD (Max)']:.4f}")
    print(f"Pearson (All): {metrics['Pearson (All)']:.4f}, Pearson (Max): {metrics['Pearson (Max)']:.4f}")
    print(f"R^2 (All): {metrics['R^2 (All)']:.4f}, R^2 (Max): {metrics['R^2 (Max)']:.4f}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
카테고리 Category 6의 데이터가 충분하지 않음

카테고리: Category 1
MAE (All): 1.5129, MAE (Max): 1.7676
MAD (All): 9.6117, MAD (Max): 8.8074
Pearson (All): 0.9857, Pearson (Max): 0.9819
R^2 (All): 0.9713, R^2 (Max): 0.9631

카테고리: Category 3
MAE (All): 1.4012, MAE (Max): 1.2176
MAD (All): 4.3929, MAD (Max): 3.7064
Pearson (All): 0.9715, Pearson (Max): 0.9731
R^2 (All): 0.9133, R^2 (Max): 0.9264

카테고리: Category 4
MAE (All): 2.2700, MAE (Max): 2.1400
MAD (All): 0.6900, MAD (Max): 0.6900
Pearson (All): -1.0000, Pearson (Max): -1.0000
R^2 (All): -19.5322, R^2 (Max): -17.1895

카테고리: Category 5
MAE (All): 1.7100, MAE (Max): 1.6240
MAD (All): 5.6640, MAD (Max): 5.0688
Pearson (All): 0.9681, Pearson (Max): 0.9740
R^2 (All): 0.9125, R^2 (Max): 0.9125

카테고리: Category 7
MAE (All): 1.8544, MAE (Max): 1.5844
MAD (All): 6.5669, MAD (Max): 6.3188
Pearson (All): 0.9376, Pearson (Max): 0.9424
R^

In [ ]:
#22개
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
finalpass_path = '/content/drive/MyDrive/category1_entries.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/la_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_LA_GT.csv'

# 1. Finalpass_entries.csv에서 'Category 2' 열의 파일 이름을 가져오기 (4자리 숫자로 매칭)
finalpass_df = pd.read_csv(finalpass_path)
finalpass_files = finalpass_df['Category 1'].str[:4]  # 파일 이름의 앞 4자리 추출

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 앞 4자리로 파일 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리

# Finalpass_entries.csv에 있는 파일만 필터링
ap_cobb_final_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(finalpass_files)]
results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(finalpass_files)]

# 4. NaN 값 처리: NaN 값이 있으면 이를 무시하고 계산
ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

# 각도 값 데이터
y_true_all = results_gt_filtered['all'].values  # GT 'all' 값
y_pred_all = ap_cobb_final_filtered['all'].values  # 예측 'all' 값

y_true_max = results_gt_filtered['max'].values  # GT 'max' 값
y_pred_max = ap_cobb_final_filtered['max'].values  # 예측 'max' 값

# 샘플 이름, GT 값, 예측 값 함께 저장할 리스트
sample_results = []
for i in range(len(y_true_all)):
    image_name = ap_cobb_final_filtered.iloc[i]['image_name']
    gt_all = y_true_all[i]
    pred_all = y_pred_all[i]
    gt_max = y_true_max[i]
    pred_max = y_pred_max[i]
    sample_results.append([image_name, gt_all, pred_all, gt_max, pred_max])

# DataFrame으로 변환하여 보기 쉽게
sample_results_df = pd.DataFrame(sample_results, columns=['Image Name', 'GT All', 'Pred All', 'GT Max', 'Pred Max'])

# 5. MAE 계산
mae_all = mean_absolute_error(y_true_all, y_pred_all)
mae_max = mean_absolute_error(y_true_max, y_pred_max)

# 6. MAD 계산 (Mean Absolute Deviation)
mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

# 7. Pearson 상관 계수
pearson_all, _ = pearsonr(y_true_all, y_pred_all)
pearson_max, _ = pearsonr(y_true_max, y_pred_max)

# 8. Spearman 상관 계수
spearman_all, _ = spearmanr(y_true_all, y_pred_all)
spearman_max, _ = spearmanr(y_true_max, y_pred_max)

# 9. MAPE 계산 (Mean Absolute Percentage Error)
mape_all = np.mean(np.abs((y_true_all - y_pred_all) / y_true_all)) * 100
mape_max = np.mean(np.abs((y_true_max - y_pred_max) / y_true_max)) * 100

# 10. R값 (결정 계수, R^2) 계산
r2_all = r2_score(y_true_all, y_pred_all)
r2_max = r2_score(y_true_max, y_pred_max)

# 결과 출력
print("0번-4번 사이 각도 비교:")
print(sample_results_df[['Image Name', 'GT All', 'Pred All']])
print(f"0번-4번 사이 각도에 대한 MAE: {mae_all:.4f}")
print(f"Pearson 상관 계수: {pearson_all:.4f}")
print(f"Spearman 상관 계수: {spearman_all:.4f}")
print(f"MAPE: {mape_all:.4f}%")
print(f"MAD: {mad_all:.4f}")
print(f"R^2 (결정 계수): {r2_all:.4f}")

print("\n최대 각도 비교:")
print(sample_results_df[['Image Name', 'GT Max', 'Pred Max']])
print(f"최대 각도에 대한 MAE: {mae_max:.4f}")
print(f"Pearson 상관 계수: {pearson_max:.4f}")
print(f"Spearman 상관 계수: {spearman_max:.4f}")
print(f"MAPE: {mape_max:.4f}%")
print(f"MAD: {mad_max:.4f}")
print(f"R^2 (결정 계수): {r2_max:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0번-4번 사이 각도 비교:
          Image Name  GT All  Pred All
0   0008-M-022Y1.jpg   22.34     21.72
1   0026-F-029Y1.jpg   35.46     34.71
2   0046-F-021Y1.jpg   31.76     32.43
3   0057-F-030Y1.jpg   28.32     25.74
4   0065-F-032Y1.jpg   31.11     32.27
5   0109-F-042Y1.jpg   21.15     22.85
6   0140-M-024Y1.jpg    0.83      5.56
7   0141-M-055Y1.jpg    3.40      2.87
8   0142-F-051Y1.jpg    6.10      7.30
9   0157-M-060Y1.jpg   28.26     25.03
10  0180-M-082Y1.jpg   13.48      9.68
11  0183-F-080Y1.jpg   16.57     15.79
12  0188-F-066Y1.jpg   34.52     35.15
13  0189-M-068Y1.jpg   12.94     12.38
14  0195-F-064Y1.jpg   29.85     30.03
15  0202-F-068Y1.jpg   33.81     33.41
16  0206-F-070Y1.jpg   43.64     42.29
17  0211-F-055Y1.jpg   21.73     18.71
18  0226-M-060Y1.jpg   30.61     32.22
19  0228-F-057Y1.jpg   38.81     40.42
20  0379-M-040Y1.jpg   27.39     26.

<ipython-input-6-5f51a19df138>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-6-5f51a19df138>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)


In [ ]:
#22개 AP
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
finalpass_path = '/content/drive/MyDrive/category1_entries.csv'
ap_cobb_final_path = '/content/drive/MyDrive/cobb angle/ap_cobb_final_plz.csv'
results_gt_path = '/content/drive/MyDrive/cobb angle/results_GT.csv'

# 1. Finalpass_entries.csv에서 'Category 2' 열의 파일 이름을 가져오기 (4자리 숫자로 매칭)
finalpass_df = pd.read_csv(finalpass_path)
finalpass_files = finalpass_df['Category 1'].str[:4]  # 파일 이름의 앞 4자리 추출

# 2. ap_cobb_final_plz.csv와 results_GT.csv 파일 불러오기
ap_cobb_final_df = pd.read_csv(ap_cobb_final_path)
results_gt_df = pd.read_csv(results_gt_path)

# 3. 앞 4자리로 파일 매칭
ap_cobb_final_df['prefix'] = ap_cobb_final_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리
results_gt_df['prefix'] = results_gt_df['image_name'].str[:4]  # 첫 번째 열의 앞 4자리

# Finalpass_entries.csv에 있는 파일만 필터링
ap_cobb_final_filtered = ap_cobb_final_df[ap_cobb_final_df['prefix'].isin(finalpass_files)]
results_gt_filtered = results_gt_df[results_gt_df['prefix'].isin(finalpass_files)]

# 4. NaN 값 처리: NaN 값이 있으면 이를 무시하고 계산
ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)

# 각도 값 데이터
y_true_all = results_gt_filtered['all'].values  # GT 'all' 값
y_pred_all = ap_cobb_final_filtered['all'].values  # 예측 'all' 값

y_true_max = results_gt_filtered['max'].values  # GT 'max' 값
y_pred_max = ap_cobb_final_filtered['max'].values  # 예측 'max' 값

# 샘플 이름, GT 값, 예측 값 함께 저장할 리스트
sample_results = []
for i in range(len(y_true_all)):
    image_name = ap_cobb_final_filtered.iloc[i]['image_name']
    gt_all = y_true_all[i]
    pred_all = y_pred_all[i]
    gt_max = y_true_max[i]
    pred_max = y_pred_max[i]
    sample_results.append([image_name, gt_all, pred_all, gt_max, pred_max])

# DataFrame으로 변환하여 보기 쉽게
sample_results_df = pd.DataFrame(sample_results, columns=['Image Name', 'GT All', 'Pred All', 'GT Max', 'Pred Max'])

# 5. MAE 계산
mae_all = mean_absolute_error(y_true_all, y_pred_all)
mae_max = mean_absolute_error(y_true_max, y_pred_max)

# 6. MAD 계산 (Mean Absolute Deviation)
mad_all = np.mean(np.abs(y_true_all - np.mean(y_true_all)))
mad_max = np.mean(np.abs(y_true_max - np.mean(y_true_max)))

# 7. Pearson 상관 계수
pearson_all, _ = pearsonr(y_true_all, y_pred_all)
pearson_max, _ = pearsonr(y_true_max, y_pred_max)

# 8. Spearman 상관 계수
spearman_all, _ = spearmanr(y_true_all, y_pred_all)
spearman_max, _ = spearmanr(y_true_max, y_pred_max)

# 9. MAPE 계산 (Mean Absolute Percentage Error)
mape_all = np.mean(np.abs((y_true_all - y_pred_all) / y_true_all)) * 100
mape_max = np.mean(np.abs((y_true_max - y_pred_max) / y_true_max)) * 100

# 10. R값 (결정 계수, R^2) 계산
r2_all = r2_score(y_true_all, y_pred_all)
r2_max = r2_score(y_true_max, y_pred_max)

# 결과 출력
print("0번-4번 사이 각도 비교:")
print(sample_results_df[['Image Name', 'GT All', 'Pred All']])
print(f"0번-4번 사이 각도에 대한 MAE: {mae_all:.4f}")
print(f"Pearson 상관 계수: {pearson_all:.4f}")
print(f"Spearman 상관 계수: {spearman_all:.4f}")
print(f"MAPE: {mape_all:.4f}%")
print(f"MAD: {mad_all:.4f}")
print(f"R^2 (결정 계수): {r2_all:.4f}")

print("\n최대 각도 비교:")
print(sample_results_df[['Image Name', 'GT Max', 'Pred Max']])
print(f"최대 각도에 대한 MAE: {mae_max:.4f}")
print(f"Pearson 상관 계수: {pearson_max:.4f}")
print(f"Spearman 상관 계수: {spearman_max:.4f}")
print(f"MAPE: {mape_max:.4f}%")
print(f"MAD: {mad_max:.4f}")
print(f"R^2 (결정 계수): {r2_max:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0번-4번 사이 각도 비교:
          Image Name  GT All  Pred All
0   0008-M-022Y0.jpg    1.85      0.22
1   0026-F-029Y0.jpg   16.27     17.59
2   0046-F-021Y0.jpg   22.24     18.99
3   0057-F-030Y0.jpg    0.98      1.45
4   0065-F-032Y0.jpg    0.71      0.78
5   0109-F-042Y0.jpg   10.50     13.57
6   0140-M-024Y0.jpg    1.54      2.05
7   0141-M-055Y0.jpg    1.59      0.48
8   0142-F-051Y0.jpg    8.56      7.78
9   0157-M-060Y0.jpg    1.25      1.60
10  0180-M-082Y0.jpg    7.78      6.50
11  0183-F-080Y0.jpg   24.08     26.58
12  0188-F-066Y0.jpg    4.12      2.90
13  0189-M-068Y0.jpg    3.25      1.97
14  0195-F-064Y0.jpg   15.51     13.34
15  0202-F-068Y0.jpg    1.27      0.26
16  0206-F-070Y0.jpg   14.50     11.58
17  0211-F-055Y0.jpg    0.82      0.38
18  0226-M-060Y0.jpg   16.42     12.57
19  0228-F-057Y0.jpg    7.35      6.39
20  0379-M-040Y0.jpg    6.67      4.

<ipython-input-7-bc153d74c99d>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ap_cobb_final_filtered.dropna(subset=['all', 'max'], inplace=True)
<ipython-input-7-bc153d74c99d>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_gt_filtered.dropna(subset=['all', 'max'], inplace=True)


In [ ]:
!pip install --upgrade pingouin